In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t_spin, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, generate_re_ortho_space_with_coeff, coeff_canonical_orthogonalization, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import solve_ritz, generate_reduced_H_non_ortho, remain_only_tridiagonal_elements, coeff_gram_schmidt
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn


In [2]:
#load integrals
#with h5py.File("data_water/eri_water.hdf5", "r") as f:
with h5py.File("/work_fast/ge49cag/code_datas/hchain/NH_10/integral.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/hchain/NH_10/u.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

#ref energy got from DMRG
e_1st_ex = -12.360857085604971
e_ground = -12.406695867918534

10
(10, 27)
(27, 27)
rl errV: 0.00023694902148543046
abs errV: 0.0007546522262828301
errt: 0.0008451096565123393
errh: 0.00013317184525868722
errht: 0.00024513110698447764


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [3]:
# #load Hamiltonian generated by above coefficients
#H_correct = scipy.io.mmread('/.mtx').tocsr()
# e1, v1 = sparse.linalg.eigsh(H_correct, which = 'SA', k = 1)
# e_ground = e1

# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)

Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [4]:
#generate thc_mpo
t_spin = get_t_spin(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t_spin)

We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [5]:
h1_spin = get_h1_spin(h1)
g_spin = get_g_spin(eri)
g_spin_phy =  g_spin.transpose(0, 2, 1, 3)
mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
print(mpo_ref.bond_dims)

[1, 4, 16, 39, 70, 87, 108, 133, 162, 195, 232, 195, 162, 133, 108, 87, 70, 39, 16, 4, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state, or even superposition of several single-excited Hatree-Fock states as initial state.

In [6]:
temp1 = generate_single_state(20, [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
#temp2 = generate_single_state(20, [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#temp3 = generate_single_state(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#temp4 = generate_single_state(20, [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

#hartree_state = add_mps(add_mps(add_mps(temp1, temp2), temp3),temp4)
#hartree_state = add_mps(temp1, temp2)
#hartree_state.orthonormalize('left')
#hartree_state.orthonormalize('right')

We generate a group of orthogonal Krylov vectors using THC-MPO, with bond dim 40 for Krylov vectors. The vectors are stored in the folder = 'foldername', thus you don't have to generate them again for next time use. 

In [7]:
N_Krylov_1 = 30
psi_original_1 = copy.deepcopy(temp1)
max_bond_Krylov_1 = 300
trunc_tol = 1e-10
foldername_1 = f"/work_fast/ge49cag/code_datas/Krylov_H10_1st_ex"
generate_krylov_space_in_disk(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1)

Make use of method proposed in https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.205119 to improve the orthogonality of Krylov vectors. 

In [8]:
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov_1, foldername_1, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov_1, foldername_1)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)
H_reduced_1 = remain_only_tridiagonal_elements(H_reduced_1)

/work_fast/ge49cag/pytenet_thc/thc_experiments/../pytenet/global_krylov_method.py:209: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc/thc_experiments/../pytenet/global_krylov_method.py:108: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = np.vdot(temp1.as_vector(), temp2.as_vector())


In [9]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov_1, e_1st_ex, mpo_ref)

(4.226991143344314+0j)
(2.993374472897127+0j)
(1.7802013778988268+0j)
(0.8903631091077067+0j)
(0.5721007440625545+0j)
(0.27780617654106443+0j)


Restart:

In [10]:
N_Krylov_2 = 30
psi_original_2 = copy.deepcopy(v_ritz_1)
max_bond_Krylov_2 = 300
trunc_tol = 0
foldername_2= f"/work_fast/ge49cag/code_datas/Krylov_H10_1st_ex_restart"
generate_krylov_space_in_disk(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original_2, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2)


In [11]:
H_reduced_non_rotho_2 = generate_reduced_H_non_ortho(N_Krylov_2, foldername_2, mpo_ref)
coeff_2 = coeff_gram_schmidt(N_Krylov_2, foldername_2)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)
H_reduced_2 = remain_only_tridiagonal_elements(H_reduced_2)

In [12]:
e_ritz_2, v_ritz_2 = solve_ritz(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_1st_ex, mpo_ref)

(0.1121501878757396+0j)
(0.07950359386698658+0j)
(0.059492427592278574+0j)
(0.05256530786707003+0j)


Restart:

In [13]:
N_Krylov_3 = 30
psi_original_3 = copy.deepcopy(v_ritz_2)
max_bond_Krylov_3 = 300
trunc_tol = 0
foldername_3 = f"/work_fast/ge49cag/code_datas/Krylov_H10_1st_ex_restart_2"
generate_krylov_space_in_disk(N_Krylov_3, H_mu_nu_list_spin_layer, psi_original_3, max_bond_Krylov_3, trunc_tol, r_thc, foldername_3)


In [21]:

H_reduced_non_rotho_3 = generate_reduced_H_non_ortho(N_Krylov_3, foldername_3, mpo_ref)
coeff_3 = coeff_gram_schmidt(N_Krylov_3, foldername_3)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_3 = np.einsum('ik, kl, jl -> ij', coeff_3.conj(), H_reduced_non_rotho_3, coeff_3)
H_reduced_3 = remain_only_tridiagonal_elements(H_reduced_3)

/work_fast/ge49cag/pytenet_thc/thc_experiments/../pytenet/global_krylov_method.py:209: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc/thc_experiments/../pytenet/global_krylov_method.py:108: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = np.vdot(temp1.as_vector(), temp2.as_vector())


In [22]:
e_ritz_3, v_ritz_3 = solve_ritz(foldername_3, H_reduced_3, N_Krylov_3, coeff_3, max_bond_Krylov_3, e_1st_ex, mpo_ref)

(0.05131517243819417+0j)
(0.05026053657298846+0j)
(0.02143690215143046+0j)


Restart:

In [6]:
#N_Krylov_4 = 10
N_Krylov_4 = 30
psi_original_4 = copy.deepcopy(v_ritz_3)
max_bond_Krylov_4 = 300
trunc_tol = 0
foldername_4 = f"/work_fast/ge49cag/code_datas/Krylov_H10_1st_ex_restart_3"
generate_krylov_space_in_disk(N_Krylov_4, H_mu_nu_list_spin_layer, psi_original_4, max_bond_Krylov_4, trunc_tol, r_thc, foldername_4)


In [7]:
H_reduced_non_rotho_4 = generate_reduced_H_non_ortho(N_Krylov_4, foldername_4, mpo_ref)
coeff_4 = coeff_gram_schmidt(N_Krylov_4, foldername_4)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_4 = np.einsum('ik, kl, jl -> ij', coeff_4.conj(), H_reduced_non_rotho_4, coeff_4)
H_reduced_4 = remain_only_tridiagonal_elements(H_reduced_4)

/work_fast/ge49cag/pytenet_thc/thc_experiments/../pytenet/global_krylov_method.py:209: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc/thc_experiments/../pytenet/global_krylov_method.py:108: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [8]:
np.linalg.eigh(H_reduced_4)
#-12.360857085604971
#-12.307896219843427

EighResult(eigenvalues=array([-12.38336765, -12.30906174, -12.03200597, -11.66910122,
       -11.21769781, -10.65635719, -10.01412948,  -9.33713061,
        -8.7753233 ,  -8.04746138,  -7.333057  ,  -6.65128275,
        -6.23798246,  -5.8965588 ,  -5.00485996]), eigenvectors=array([[-0.93628837, -0.29449794,  0.12598213,  0.08559016, -0.07353495,
         0.0541769 , -0.04684395, -0.04001676,  0.03024498, -0.01309335,
        -0.00954231,  0.00724075, -0.00738186,  0.003131  ,  0.00238415],
       [ 0.15610431, -0.03391823,  0.1469279 ,  0.21765906, -0.31293243,
         0.34592819, -0.41323986, -0.45579117,  0.4089537 , -0.21319542,
        -0.18123721,  0.15625173, -0.17087142,  0.07653031,  0.06634049],
       [-0.1297454 ,  0.1192484 , -0.27340069, -0.33030358,  0.3657309 ,
        -0.26188268,  0.12178076, -0.08611941,  0.24084129, -0.23581837,
        -0.29235067,  0.32758392, -0.40828837,  0.20138045,  0.21647259],
       [ 0.1220961 , -0.18582561,  0.3385703 ,  0.31999946, -0.2

In [9]:
e_ritz_4, v_ritz_4 = solve_ritz(foldername_4, H_reduced_4, N_Krylov_4, coeff_4, max_bond_Krylov_4, e_1st_ex, mpo_ref)

(-0.009253973323286147+0j)
(-0.01577341801575116+0j)
(-0.019837704735463646+0j)


In [ ]:
#error due to thc?
#(-0.00036100625602308867+0j)

In [10]:
max_index = np.argmax((v_ritz_4.as_vector()))
#v_ritz_2.as_vector()[max_index]
print(bin(max_index))

0b10011001100110011001


In [ ]:
#可以用number op统计自旋